In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [3]:
# Load the dataset
df = pd.read_csv('..//..//dataset//final_dataset.csv')  


# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X


# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [4]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [ ]:
def build_lstm():
    model = Sequential()
    
    # Input layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    # LSTM layer with 3 hidden layers
    model.add(LSTM(units=50, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model


In [ ]:
# Instantiate and train the model
model = build_lstm()
import time
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    verbose=1
)
end_time = time.time()

Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 0.1840 - mae: 0.1368 - mse: 0.0623 - val_loss: 5.8858e-05 - val_mae: 5.8855e-05 - val_mse: 2.0290e-08
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 8.7780e-05 - mae: 4.7157e-05 - mse: 3.7846e-06 - val_loss: 1.7436e-05 - val_mae: 1.7433e-05 - val_mse: 1.3246e-09
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 3.7890e-05 - mae: 1.6922e-05 - mse: 1.8652e-06 - val_loss: 9.4711e-06 - val_mae: 9.4721e-06 - val_mse: 2.4509e-10
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 1.2136e-05 - mae: 8.4669e-06 - mse: 4.7216e-07 - val_loss: 7.1260e-06 - val_mae: 7.1288e-06 - val_mse: 1.8659e-10
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 1.3087e-04 - mae: 1.8981e-05 - mse: 1.0037e-05 - val_loss: 3.9088e-06 - val_mae: 3.9058e-06 - val_mse: 3.1784e-11
Epoch 6/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 6.0890e-05 - mae: 2.0595e-05 - mse: 8.3619e-06 - val_loss: 2.5187e-06

KeyboardInterrupt: 

In [ ]:
# Calculate total training time
total_training_time = end_time - start_time  # in seconds
epochs = 50 # e.g., 50
steps_per_epoch = 2500# e.g., 2000

# Total steps
total_steps = epochs * steps_per_epoch

# Calculate time per step
time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
#micro seconds
time_per_step_us = time_per_step_ms * 1000

# Calculate time per epoch
time_per_epoch = total_training_time / epochs

# Display results
print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Steps: 50
Time per Step: 2.789492 sec (2789.49 ms)
Time per Step: 2789491.94 µs
Time per Epoch: 2.79 sec


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')


#ACCUARCY
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100}')


625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Mean Squared Error for y_pred (MSE): 0.1132557466626167
Root Mean Squared Error (RMSE): 0.3365349115063944
Accuracy: 98.625


In [ ]:
import matplotlib.pyplot as plt

# Visualizing the loss, MSE, and MAE
plt.figure(figsize=(15, 5))

# Loss plot
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# MSE plot
plt.subplot(1, 3, 2)
plt.plot(history.history['mse'], label='Train MSE')
plt.plot(history.history['val_mse'], label='Validation MSE')
plt.title('Mean Squared Error')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# MAE plot
plt.subplot(1, 3, 3)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()
